# Working with Twitter in Python

In this tutorial we will be using the [tweepy](http://www.tweepy.org/) API for twitter.

To install it run:
* pip install tweepy

Next we need to register with Twitter
* Go to [https://dev.twitter.com](https://dev.twitter.com)
* Click on "Manage Your Apps" and create a new App
* If your project doesn't have a website yet, just use http://example.com
* For this tutorial, we do not need an Oauth token since we just care about scraping Twitter data

Now we have a consumer API token, but we need secret tokens for our app
* Click on Keys and Access tokens
* Then click on the "Create my access token button"

# Create a token file (twitter.cfg)

Add in your keys so the file looks like:

[api]

consumer_key = %consumer_key%

consumer_secret = %consumer_secret%

access_token = %access_token%

access_token_secret = %access_token_secret%

In [ ]:
from pprint import pprint
import tweepy
import configparser
import json

parser = configparser.ConfigParser()
parser.read(['twitter.cfg'])
consumer_key = parser.get('api', 'consumer_key')
consumer_secret = parser.get('api', 'consumer_secret')
access_token = parser.get('api', 'access_token')
access_token_secret = parser.get('api', 'access_token_secret')

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# Getting Trending HashTags
# Based on https://woe.spum.org/ (e.g. search United States)
trends = api.trends_place(23424977)
pprint(trends)

In [ ]:
trends_list = [i['name'] for i in trends[0]['trends']]
print trends_list

# Now let's set up streaming!

In [ ]:
from tweepy.streaming import StreamListener

In [ ]:
from textblob import TextBlob
class StdOutStreamer(StreamListener):
    def __init__(self):
        super(StdOutStreamer, self).__init__()
        self.tweet_cnt = 0
        self.retweet_cnt = 0

    def on_data(self, data):
        data = json.loads(data)
        # Check if there is text tied to the tweet
        if 'text' in data:
            print TextBlob(data['text']).sentiment
            if data['text'].strip().startswith('RT'):
                self.retweet_cnt += 1
            else:
                self.tweet_cnt += 1
        # Just to exit
        if self.tweet_cnt >= 100:
            raise Exception("Too many tweets")
    def on_error(self, status):
        print status
        print "Tweet Cnt: %d, RT Cnt: %d"%(self.retweet_cnt, self.tweet_cnt)

In [ ]:
try:
    l = StdOutStreamer()
    stream = tweepy.Stream(auth, l)
    stream.filter(track=trends_list)
except KeyboardInterrupt, e:
    pass
except Exception, e:
    pass

# Downloading Tweets from a User

In [ ]:
tweets = []
times = []
username = 'AppleSupport'
new_tweets = api.user_timeline(screen_name=username, count=200)
tweets.extend([i.text for i in new_tweets])
times.extend([i.created_at for i in new_tweets])


while len(new_tweets) > 0:
    oldest = new_tweets[-1].id - 1
    new_tweets = api.user_timeline(screen_name=username, count=200, max_id=oldest)
    
    tweets.extend([i.text for i in new_tweets])
    times.extend([i.created_at for i in new_tweets])

In [ ]:
#print tweets[2]
for i, tweet in enumerate(tweets):
    print times[i], TextBlob(tweet).sentiment